In [1]:
url = "http://www.nasdaq.com/earnings/earnings-calendar.aspx" #?date=2017-Nov-06"
frame = '<iframe src=' + url + 'width=700 height=500></iframe>'

In [2]:
from bs4 import BeautifulSoup
import urllib
r = urllib.urlopen(url).read()
soup = BeautifulSoup(r)
print type(soup)

C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\ProgramData\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


<class 'bs4.BeautifulSoup'>


In [3]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html class="inner no-js" lang="en-us" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-K2BQVP7');
  </script>
  <!-- End Google Tag Manager -->
  <script>
   (function(){
var is_chrome;
if(typeof navigator.vendor!="undefined")
	is_chrome = ((navigator.userAgent.toLowerCase().indexOf('chrome') > -1) &&(navigator.vendor.toLowerCase().indexOf("google") > -1));
else
	is_chrome = false;
if(is_chrome)
{
//instart
//copyright Wed Oct 25 2017 - version: 8.3.16
!function(t){if(function(){var n=!0;try{var i="object"==typeof t.IXC_298_004268222022440815&&t.IXC_298


In [4]:
companiesTable = soup.find(id='ECCompaniesTable')
print(type(companiesTable))

<class 'bs4.element.Tag'>


In [5]:
from enum import Enum
class Hours(Enum):
    DAY = 1
    NIGHT = 2
    UNKNOWN = 3

In [6]:
rows = companiesTable.findChildren(['th', 'tr'])

In [7]:
def parse_time_col(time_child):
    if len(time_child.select('a')) == 0:
        return Hours.UNKNOWN
    
    title = time_child.select('a')[0]['title']
    if title == 'After Hours Quotes':
        return Hours.NIGHT
    if title == 'Pre-market Quotes':
        return Hours.DAY
    return Hours.UNKNOWN

In [8]:
from re import sub
from decimal import Decimal

def parse_decimal(money):
    if money == '$n/a':
        return None
    return Decimal(money[1:])

In [9]:
import pandas as pd
from datetime import datetime

def parse_table():
    column_titles = ['Time', 'Symbol', 'Reported Date', 'Quarter', 'ConsensusEPS', '# of Ests', 'Unknown', 'LastYearEPS', 'PercentSurprise']
    df = pd.DataFrame(columns=column_titles)

    row_marker = 0
    for row in rows:
        children = row.findChildren('td')
        if len(children) == 0:
            continue
            
        row_data = [column.get_text().strip() for column in children]
        row_data[0] = parse_time_col(children[0])
        row_data[2] = datetime.strptime(row_data[2], '%m/%d/%Y')
        row_data[4] = parse_decimal(row_data[4])
        row_data[5] = int(row_data[5])
        if row_data[6] == 'n/a':
            row_data[6] = None
        else:
            row_data[6] = datetime.strptime(row_data[6], '%m/%d/%Y')
        row_data[7] = parse_decimal(row_data[7])
            
        df.loc[row_marker] = row_data
        row_marker += 1

    return df

In [10]:
df = parse_table()
df.tail()

Time                                           Symbol  \
373  Hours.NIGHT        Wingstop Inc. (WING) Market Cap: $984.68M   
374  Hours.NIGHT      Xperi Corporation (XPER) Market Cap: $1.14B   
375  Hours.NIGHT  YRC Worldwide, Inc. (YRCW) Market Cap: $451.68M   
376    Hours.DAY      Yum! Brands, Inc. (YUM) Market Cap: $25.67B   
377    Hours.DAY            Zoetis Inc. (ZTS) Market Cap: $31.22B   

    Reported Date   Quarter ConsensusEPS # of Ests              Unknown  \
373    2017-11-02  Sep 2017         0.15         8  2016-11-01 00:00:00   
374    2017-11-02  Sep 2017         0.36         1  2016-11-01 00:00:00   
375    2017-11-02  Sep 2017         0.28         3  2016-10-27 00:00:00   
376    2017-11-02  Sep 2017         0.66        10  2016-10-05 00:00:00   
377    2017-11-02  Sep 2017         0.63         9  2016-11-02 00:00:00   

    LastYearEPS PercentSurprise  
373        0.13             Met  
374        0.53             Met  
375        0.42             Met  
376        1.09             Met  
377        0.52             Met

In [11]:
filtered_df = df[(df['Time'] == Hours.NIGHT) & (df['ConsensusEPS'] > df['LastYearEPS'])]
filtered_df

Time                                             Symbol  \
3    Hours.NIGHT  Accelerate Diagnostics, Inc. (AXDX) Market Cap...   
10   Hours.NIGHT  Adtalem Global Education Inc. (ATGE) Market Ca...   
13   Hours.NIGHT    Aethlon Medical, Inc. (AEMD) Market Cap: $14.8M   
18   Hours.NIGHT  Alliant Energy Corporation (LNT) Market Cap: $10B   
19   Hours.NIGHT  Allscripts Healthcare Solutions, Inc. (MDRX) M...   
20   Hours.NIGHT  Alpha and Omega Semiconductor Limited (AOSL) M...   
25   Hours.NIGHT     American Homes 4 Rent (AMH) Market Cap: $5.54B   
27   Hours.NIGHT  American Vanguard Corporation (AVD) Market Cap...   
34   Hours.NIGHT             Apple Inc. (AAPL) Market Cap: $873.13B   
35   Hours.NIGHT  Apricus Biosciences, Inc. (APRI) Market Cap: $...   
36   Hours.NIGHT  Aratana Therapeutics, Inc. (PETX) Market Cap: ...   
41   Hours.NIGHT    Arista Networks, Inc. (ANET) Market Cap: $14.5B   
45   Hours.NIGHT           Ashford Inc. (AINC) Market Cap: $145.57M   
54   Hours.NIGHT  Axcelis Technologies, Inc. (ACLS) Market Cap: ...   
56   Hours.NIGHT  Basic Energy Services, Inc. (BAS) Market Cap: ...   
63   Hours.NIGHT   Black Hills Corporation (BKH) Market Cap: $3.49B   
66   Hours.NIGHT  Boingo Wireless, Inc. (WIFI) Market Cap: $937.75M   
68   Hours.NIGHT  Bottomline Technologies, Inc. (EPAY) Market Ca...   
71   Hours.NIGHT          CafePress Inc. (PRSS) Market Cap: $34.46M   
74   Hours.NIGHT  Capstone Turbine Corporation (CPST) Market Cap...   
75   Hours.NIGHT  Cara Therapeutics, Inc. (CARA) Market Cap: $40...   
76   Hours.NIGHT        Carbonite, Inc. (CARB) Market Cap: $631.04M   
79   Hours.NIGHT  Career Education Corporation (CECO) Market Cap...   
83   Hours.NIGHT          CBS Corporation (CBS) Market Cap: $22.54B   
90   Hours.NIGHT    Chemours Company (The) (CC) Market Cap: $10.46B   
102  Hours.NIGHT             Cohu, Inc. (COHU) Market Cap: $721.36M   
103  Hours.NIGHT  Computer Programs and Systems, Inc. (CPSI) Mar...   
108  Hours.NIGHT   Consolidated Edison Inc (ED) Market Cap: $26.31B   
110  Hours.NIGHT  Corcept Therapeutics Incorporated (CORT) Marke...   
111  Hours.NIGHT         CryoPort, Inc. (CYRX) Market Cap: $195.06M   
..           ...                                                ...   
283  Hours.NIGHT      QCR Holdings, Inc. (QCRH) Market Cap: $629.4M   
288  Hours.NIGHT    RE/MAX Holdings, Inc. (RMAX) Market Cap: $1.18B   
289  Hours.NIGHT             RealPage, Inc. (RP) Market Cap: $3.58B   
291  Hours.NIGHT  Republic Services, Inc. (RSG) Market Cap: $21.91B   
292  Hours.NIGHT  Research Frontiers Incorporated (REFR) Market ...   
295  Hours.NIGHT         Rice Energy Inc. (RICE) Market Cap: $6.06B   
296  Hours.NIGHT  Rice Midstream Partners LP (RMP) Market Cap: $...   
297  Hours.NIGHT        Rogers Corporation (ROG) Market Cap: $2.77B   
303  Hours.NIGHT  SeaSpine Holdings Corporation (SPNE) Market Ca...   
304  Hours.NIGHT  Second Sight Medical Products, Inc. (EYES) Mar...   
305  Hours.NIGHT  Select Medical Holdings Corporation (SEM) Mark...   
310  Hours.NIGHT  Sierra Wireless, Inc. (SWIR) Market Cap: $723.33M   
311  Hours.NIGHT          SM Energy Company (SM) Market Cap: $2.38B   
313  Hours.NIGHT     Spark Energy, Inc. (SPKE) Market Cap: $484.83M   
317  Hours.NIGHT          SPX Corporation (SPXC) Market Cap: $1.25B   
318  Hours.NIGHT    Stag Industrial, Inc. (STAG) Market Cap: $2.52B   
328  Hours.NIGHT    T2 Biosystems, Inc. (TTOO) Market Cap: $133.52M   
329  Hours.NIGHT   Tableau Software, Inc. (DATA) Market Cap: $6.41B   
330  Hours.NIGHT  Tallgrass Energy GP, LP (TEGP) Market Cap: $3.93B   
331  Hours.NIGHT  Tallgrass Energy Partners, LP (TEP) Market Cap...   
351  Hours.NIGHT               Trinseo S.A. (TSE) Market Cap: $3.1B   
352  Hours.NIGHT        Trupanion, Inc. (TRUP) Market Cap: $844.67M   
354  Hours.NIGHT  Turtle Beach Corporation (HEAR) Market Cap: $3...   
357  Hours.NIGHT    UMH Properties, Inc. (UMH) Market Cap: $506.39M   
360  Hours.NIGHT  Universal Display 

In [12]:
symbols = [s[s.find("(")+1:s.find(")")] for s in filtered_df['Symbol']]
print len(symbols)

95


In [13]:
symbols

[u'AXDX',
 u'ATGE',
 u'AEMD',
 u'LNT',
 u'MDRX',
 u'AOSL',
 u'AMH',
 u'AVD',
 u'AAPL',
 u'APRI',
 u'PETX',
 u'ANET',
 u'AINC',
 u'ACLS',
 u'BAS',
 u'BKH',
 u'WIFI',
 u'EPAY',
 u'PRSS',
 u'CPST',
 u'CARA',
 u'CARB',
 u'CECO',
 u'CBS',
 u'The',
 u'COHU',
 u'CPSI',
 u'ED',
 u'CORT',
 u'CYRX',
 u'SAUC',
 u'EGLE',
 u'ELON',
 u'ERI',
 u'ECPG',
 u'EGL',
 u'EOG',
 u'PLUS',
 u'EVH',
 u'FDUS',
 u'FNGN',
 u'FLDM',
 u'FTAI',
 u'GNW',
 u'GTE',
 u'HDP',
 u'XENT',
 u'INTT',
 u'JCOM',
 u'LADR',
 u'LNTH',
 u'LYV',
 u'MCHX',
 u'MATX',
 u'MTD',
 u'MSI',
 u'MRC',
 u'OMED',
 u'OEC',
 u'OVAS',
 u'PCTY',
 u'PDLI',
 u'PBA',
 u'PKI',
 u'PTCT',
 u'QCRH',
 u'RMAX',
 u'RP',
 u'RSG',
 u'REFR',
 u'RICE',
 u'RMP',
 u'ROG',
 u'SPNE',
 u'EYES',
 u'SEM',
 u'SWIR',
 u'SM',
 u'SPKE',
 u'SPXC',
 u'STAG',
 u'TTOO',
 u'DATA',
 u'TEGP',
 u'TEP',
 u'TSE',
 u'TRUP',
 u'HEAR',
 u'UMH',
 u'OLED',
 u'UQM',
 u'USAK',
 u'VECO',
 u'WLDN',
 u'WING']

In [14]:
filtered_df.describe()

Time                                    Symbol  \
count            95                                        95   
unique            1                                        95   
top     Hours.NIGHT  MRC Global Inc. (MRC) Market Cap: $1.62B   
freq             95                                         1   
first           NaN                                       NaN   
last            NaN                                       NaN   

              Reported Date   Quarter ConsensusEPS  # of Ests  \
count                    95        95           95       95.0   
unique                    1         1           77       12.0   
top     2017-11-02 00:00:00  Sep 2017        -0.09        2.0   
freq                     95        95            3       23.0   
first   2017-11-02 00:00:00       NaN          NaN        NaN   
last    2017-11-02 00:00:00       NaN          NaN        NaN   

                    Unknown LastYearEPS PercentSurprise  
count                    94          95              95  
unique                   12          76               1  
top     2016-11-03 00:00:00        0.11             Met  
freq                     41           3              95  
first                   NaN         NaN             NaN  
last                    NaN         NaN             NaN